In [1]:
from LogisticDecoder import LogisticDecoder
from common import clean_descriptions, samples_to_dict, VALIDATION_FILENAMES, corpus_bleu_score
from RNNDecoder import RNNModel

In [2]:
''' 
get the actual rows from the df corresponding to the different sets
'''
# get the samples with the given filenames
small_train_samples = clean_descriptions('../data/flickr_8k/small_train.csv')
validation_samples = clean_descriptions('../data/flickr_8k/validation.csv')

# NOTE uncomment as needed
# train_samples = clean_descriptions('../data/flickr_8k/train.csv')
# test_samples = clean_descriptions('../data/flickr_8k/test.csv')
# train_and_val_samples = clean_descriptions('../data/flickr_8k/train_and_val.csv')


small_train_dict = samples_to_dict(small_train_samples)
validation_dict = samples_to_dict(validation_samples)
val_captions = list(validation_dict.values())

In [3]:
'''
create logistic regression model
get bleu score on validation set
'''
logistic_decoder = LogisticDecoder(15, small_train_samples)
# logistic_decoder.fit(small_train_dict, 5, '../models/compare_3_models/LogisticModel',verbose=True)

In [4]:
logistic_decoder.load('../models/compare_3_models/LogisticModel')

Model loaded from ../models/compare_3_models/LogisticModel


In [6]:
logistic_captions = logistic_decoder.generate_captions_for_files(VALIDATION_FILENAMES, verbose=False)

In [7]:
logistic_bleu_scores = corpus_bleu_score(val_captions, logistic_captions)

c:\Users\toyso\anaconda3\envs\tf-gpu\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


BLEU-1: 0.29785561917592907
BLEU-2: 0.07837888206846033
BLEU-3: 0.01807101168302499
BLEU-4: 6.019673799174596e-79


In [3]:

'''
load RNN without dropout

get bleu score on validation set
'''
rnn_without_dropout_model = RNNModel(False, small_train_samples)
rnn_without_dropout_model.train_save_model(input_dict=small_train_dict, save_path='../models/compare_3_models/RNN_without_dropout', epochs=5)



The top 30 sequence lengths are:
[35, 35, 34, 33, 33, 33, 33, 32, 31, 31, 31, 31, 30, 30, 30, 30, 30, 30, 29, 29, 29, 29, 29, 29, 29, 29, 28, 28, 28, 28]
The longest sequence length from the training and validation samples is 35
The average sequence length from the training and validation samples is 12
16180/16180 [==============================] - 368s 23ms/step - loss: 3.3876


c:\Users\toyso\anaconda3\envs\tf-gpu\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


In [ ]:
# rnn_without_dropout_model.load('../models/compare_3_models/RNN_without_dropout')

In [4]:
without_dropout_captions = rnn_without_dropout_model.generate_captions_for_files(VALIDATION_FILENAMES, verbose=False)

In [5]:
without_dropout_bleu_scores = corpus_bleu_score(val_captions, without_dropout_captions)

BLEU-1: 0.3256668642560759
BLEU-2: 0.12261347302117036
BLEU-3: 0.03905333234637138
BLEU-4: 0.009979650225174266


In [3]:
'''
load RNN with dropout layers

get bleu score on validation set
'''
rnn_with_dropout_model = RNNModel(True, small_train_samples)
# rnn_with_dropout_model.train_save_model(input_dict=small_train_dict, save_path='../models/compare_3_models/RNN_with_dropout', epochs=5)


The top 30 sequence lengths are:
[35, 35, 34, 33, 33, 33, 33, 32, 31, 31, 31, 31, 30, 30, 30, 30, 30, 30, 29, 29, 29, 29, 29, 29, 29, 29, 28, 28, 28, 28]
The longest sequence length from the training and validation samples is 35
The average sequence length from the training and validation samples is 12


In [4]:
rnn_with_dropout_model.load('../models/compare_3_models/RNN_with_dropout')

model loaded successfully!


In [5]:
with_dropout_captions = rnn_with_dropout_model.generate_captions_for_files(VALIDATION_FILENAMES, verbose=False)

In [6]:
with_dropout_bleu_scores = corpus_bleu_score(val_captions, with_dropout_captions)

BLEU-1: 0.330918147854636
BLEU-2: 0.12436767922869052
BLEU-3: 0.04006884227079066
BLEU-4: 0.011906023055355095


In [ ]:
'''
Note:

best model was the model with dropout layers

now do hyperparameter tuning on best model
'''

adam_5_epochs = RNNModel(True, small_train_samples, optimizer='adam')
adam_5_epochs.train_save_model(input_dict=small_train_dict, save_path='../models/hyperparameter_tuning/rnn_5_epochs_adam', epochs=5)
# adam_5_epochs.load('../models/hyperparameter_tuning/rnn_5_epochs_adam')

In [ ]:
adam_5_captions = adam_5_epochs.generate_captions_for_files(VALIDATION_FILENAMES, verbose=False)

In [ ]:
adam_5_bleu_scores = corpus_bleu_score(val_captions, adam_5_captions)

In [ ]:
adam_10_epochs = RNNModel(True, small_train_samples, optimizer='adam')
adam_10_epochs.train_save_model(input_dict=small_train_dict, save_path='../models/hyperparameter_tuning/rnn_10_epochs_adam', epochs=10)
# adam_10_epochs.load('../models/hyperparameter_tuning/rnn_10_epochs_adam')

In [ ]:
adam_10_captions = adam_10_epochs.generate_captions_for_files(VALIDATION_FILENAMES, verbose=False)

In [ ]:
adam_10_bleu_scores = corpus_bleu_score(val_captions, adam_10_captions)

In [ ]:
sgd_5_epochs = RNNModel(True, small_train_samples, optimizer='sgd')
sgd_5_epochs.train_save_model(input_dict=small_train_dict, save_path='../models/hyperparameter_tuning/rnn_5_epochs_sgd', epochs=5)
# sgd_5_epochs.load('../models/hyperparameter_tuning/rnn_5_epochs_sgd')

In [ ]:
sgd_5_captions = sgd_5_epochs.generate_captions_for_files(VALIDATION_FILENAMES, verbose=False)

In [ ]:
sgd_5_bleu_scores = corpus_bleu_score(val_captions, sgd_5_captions)

In [ ]:
sgd_10_epochs = RNNModel(True, small_train_samples, optimizer='sgd')
sgd_10_epochs.train_save_model(input_dict=small_train_dict, save_path='../models/hyperparameter_tuning/rnn_10_epochs_sgd', epochs=10)
# sgd_10_epochs.load('../models/hyperparameter_tuning/rnn_10_epochs_sgd')

In [ ]:
sgd_10_captions = sgd_10_epochs.generate_captions_for_files(VALIDATION_FILENAMES, verbose=False)

In [ ]:
sgd_10_bleu_scores = corpus_bleu_score(val_captions, sgd_10_captions)